In [ ]:
{-# LANGUAGE FunctionalDependencies #-}
{-# LANGUAGE MonomorphismRestriction #-}
{-# LANGUAGE FlexibleContexts #-}
{-# LANGUAGE FlexibleInstances #-}

import Control.Arrow
import Control.Monad.State
import Numeric.LinearAlgebra
import Numeric.LinearAlgebra.Data
import Numeric
import Prelude hiding ((<>))

\begin{eqnarray}
Sigmoid \nonumber \\
  h(x) &=& \frac
  {1}
  {1 + \exp(-x)} \\
\nonumber \\
ReLU \nonumber \\
  h(x) &=& \begin{cases}
  x & (x > 0) \\
  0 & (x \le 0)
  \end{cases}
\end{eqnarray}

In [ ]:
sigmoid :: (Floating a) => a -> a
sigmoid a = fromIntegral 1 / (fromIntegral 1 + exp (-a))

sigmoidm :: (Floating a, Container Matrix a) => Matrix a -> Matrix a
sigmoidm = cmap sigmoid

sigmoidBackward :: (Floating a, Num (Vector a), Container Matrix a) => Matrix a -> Matrix a -> Matrix a
sigmoidBackward y d = d * (cmap (\a -> fromIntegral 1 - a) y) * y

relu :: (Ord a, Num a) => a -> a
relu = max $ fromIntegral 0

relum :: (Ord a, Num a, Container Matrix a) => Matrix a -> Matrix a
relum = cmap relu

relumBackward :: (Ord a, Num a, Num (Matrix a), Container Vector a, Container Matrix a) => Matrix a -> Matrix a -> Matrix a
relumBackward x d = d * mask
    where mask = cmap (\a -> fromIntegral $ if (0 < a) then 1 else 0) x

\begin{eqnarray}
Cross Entropy \nonumber \\
  L &=& -\displaystyle \sum_{i=1}^{n}t_i \log (y_i + d) \\
  t &:& 教師ラベル \nonumber \\
  d &:& 微小値 \nonumber \\
 \nonumber \\
Softmax \nonumber \\
  y_k &=& \frac
  {\exp(a_k - \hat{a})}
  {\displaystyle \sum_{i=1}^{n}\exp(a_i - \hat{a})} \\
  \hat{a} &=& \max \{ a_{1...n} \} \nonumber
\end{eqnarray}


In [ ]:
softmax :: (Floating a, Container Vector a) => Vector a -> Vector a
softmax v = cmap (/s) v'
    where
    m = maxElement v
    v' = cmap (\a -> exp (a - m)) v
    s = sumElements v'

softmaxm :: (Floating a, Container Vector a) => Matrix a -> Matrix a
softmaxm m = fromRows $ map softmax $ toRows m

crossEntropy :: (Floating a, Num (Vector a), Container Vector a) => Vector a -> Vector a -> a
crossEntropy t y = -(sumElements $ cmap log (y + d) * t)
    where d = 1e-10

crossEntropym :: (Floating a, Num (Vector a), Container Vector a) => Matrix a -> Matrix a -> a
crossEntropym t m = sum vs / batchSize
    where 
    vs = uncurry crossEntropy `map` (toRows t `zip` toRows m)
    batchSize = fromIntegral $ rows t

softmaxWithCross :: (Floating a, Num (Vector a), Container Vector a) => Matrix a -> Matrix a -> a
softmaxWithCross t = crossEntropym t . softmaxm

softmaxWithCrossBackward :: (Floating a, Num (Vector a), Container Vector a) => Matrix a -> Matrix a -> Matrix a
softmaxWithCrossBackward t y = (y - t) / batchSize
    where batchSize = fromIntegral $ rows t

In [ ]:
affinem :: (Floating a, Numeric a, Num (Vector a)) => Matrix a -> Vector a -> Matrix a -> Matrix a
affinem w b x = x <> w + b'
    where b' = fromColumns $ replicate (rows x) b

affinemBackward :: (Floating a, Numeric a) => Matrix a -> Matrix a -> Matrix a -> (Matrix a, Matrix a, Vector a)
affinemBackward w x d = (dx, dw, db)
    where
    dx = d <> tr w
    dw = tr x <> d
    db = fromList $ sumElements `map` toColumns d

In [ ]:
data ForwardLayer a =
    AffineForward (Matrix a) (Vector a)
  | SigmoidForward
  | ReLUForward
  | JoinedForwardLayer (ForwardLayer a) (ForwardLayer a)

infixr 4 ~>
(~>) :: ForwardLayer a -> ForwardLayer a -> ForwardLayer a
a ~> b = JoinedForwardLayer a b

data OutputLayer a = SoftmaxWithCrossForward

data ForwardNN a = ForwardNN (ForwardLayer a) (OutputLayer a)

data BackwardLayer a = 
    AffineBackward (Matrix a) (Vector a) (Matrix a)
  | SigmoidBackward (Matrix a)
  | ReLUBackward (Matrix a)
  | JoinedBackwardLayer (BackwardLayer a) (BackwardLayer a)

infixl 4 <~
(<~) :: BackwardLayer a -> BackwardLayer a -> BackwardLayer a
a <~ b = JoinedBackwardLayer a b

data BackputLayer a = SoftmaxWithCrossBackward (Matrix a) (Matrix a)

data BackwardNN a = BackwardNN (BackwardLayer a) (BackputLayer a)

In [ ]:
forward :: (Ord a, Floating a, Numeric a, Num (Vector a)) => ForwardLayer a -> Matrix a -> (BackwardLayer a, Matrix a)
forward (AffineForward w b) x = (AffineBackward w b x, affinem w b x)
forward SigmoidForward x = let y = sigmoidm x in (SigmoidBackward y, y)
forward ReLUForward x = (ReLUBackward x, relum x)
forward (JoinedForwardLayer a b) x0 = (a' <~ b', x2)
    where
    (a', x1) = forward a x0
    (b', x2) = forward b x1

backward :: (Ord a, Floating a, Numeric a, Num (Vector a)) => BackwardLayer a -> Matrix a -> (ForwardLayer a, Matrix a)
backward (AffineBackward w b x) d =  (AffineForward (w + w') (b + b'), x')
    where (x', w', b') = affinemBackward w x d
backward (SigmoidBackward y) d = (SigmoidForward, sigmoidBackward y d)
backward (ReLUBackward x) d = (ReLUForward, relumBackward x d)
backward (JoinedBackwardLayer a b) d0 = (a' ~> b', d2)
    where
    (b', d1) = backward b d0
    (a', d2) = backward a d1

output :: (Floating a, Numeric a, Num (Vector a)) => OutputLayer a -> Matrix a -> Matrix a -> (BackputLayer a, a)
output SoftmaxWithCrossForward t y = (SoftmaxWithCrossBackward t y, softmaxWithCross t y)

backput :: (Floating a, Numeric a, Num (Vector a)) => BackputLayer a -> (OutputLayer a, Matrix a)
backput (SoftmaxWithCrossBackward t x) = (SoftmaxWithCrossForward, softmaxWithCrossBackward t x)

learnForward :: (Ord a, Floating a, Numeric a, Num (Vector a)) => ForwardNN a -> Matrix a -> Matrix a -> (BackwardNN a, a)
learnForward (ForwardNN layers loss) t x = (BackwardNN layers' loss', result)
    where
    (layers', y) = forward layers x
    (loss', result) = output loss t y

learnBackward :: (Ord a, Floating a, Numeric a, Num (Vector a)) => BackwardNN a -> ForwardNN a
learnBackward  (BackwardNN layers loss) = ForwardNN layers' loss'
    where
    (loss', d) = backput loss
    (layers', _) = backward layers d

learn :: (Ord a, Floating a, Numeric a, Num (Vector a)) => ForwardNN a -> Matrix a -> Matrix a -> (ForwardNN a, a)
learn a t x = first learnBackward $ learnForward a t x

learnAll :: (Ord a, Floating a, Numeric a, Num (Vector a)) => ForwardNN a -> [(Matrix a, Matrix a)] -> ForwardNN a
learnAll = foldr f
    where f a (t, x) = fst $ learn a t x

In [ ]:
:info foldr

In [ ]:
xs = ((2><2)[1..]) :: Matrix R
ys = ((2><2)[1..]) :: Matrix R
:type xs
:type ys
print xs
print ys
print $ xs / 2.0